In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.jars", "/opt/spark/postgresql-42.7.8.jar") \
    .getOrCreate()

# Load data from Postgres
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://host.docker.internal:5432/postgres") \
    .option("dbtable", "processed_table") \
    .option("user", "postgres") \
    .option("password", "password") \
    .load()

In [10]:
fractions = {"GALAXY": 0.1, "STAR": 0.1, "QSO": 0.1}
sampled_df = df.stat.sampleBy("class", fractions, seed=42)

In [11]:
sampled_df.coalesce(1) \
    .write \
    .option("header", "true") \
    .csv("processed_data_sample", mode="overwrite")

import os 
for fname in os.listdir("processed_data_sample"):
    if fname.startswith("part-") and fname.endswith(".csv"):
        os.rename(os.path.join("processed_data_sample", fname), "processed_data_sample.csv")
        break
import shutil
shutil.rmtree("processed_data_sample")

25/12/28 17:50:29 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
25/12/28 17:50:29 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
25/12/28 17:50:29 INFO SQLHadoopMapReduceCommitProtocol: Using output committer class org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter
25/12/28 17:50:29 INFO CodeGenerator: Code generated in 43.449475 ms
25/12/28 17:50:29 INFO SparkContext: Starting job: csv at <unknown>:0
25/12/28 17:50:29 INFO DAGScheduler: Got job 1 (csv at <unknown>:0) with 1 output partitions
25/12/28 17:50:29 INFO DAGScheduler: Final stage: ResultStage 1 (csv at <unknown>:0)
25/12/28 17:50:29 INFO DAGScheduler: Parents of final stage: List()
25/12/28 17:50:29 INFO DAGScheduler: Missing parents: List()
25/12/28 17:50:29 INFO DAGScheduler: Submitting ResultStage 1 (MapPartitionsRDD[7] at csv at <unknown>:0), which has no missing parents
25/12/28 17:50:30 INFO Memo